In [24]:
import re

In [53]:
def dump_to_csv(dump_filename):
    begin_regex = re.compile('^COPY.* stdin;$')
    header_regex = re.compile(', ')
    line_regex = re.compile('\t+')
    end_regex = re.compile('^\\\.$')
    line_count = 0
    fast_forward = True
    tables = []
    with open(dump_filename, 'r') as f:
        for line in f:
            # check if line is the COPY statement (similar to INSERT)
            if re.match(begin_regex, line):
                target_table = line[5:line.find('(')-1]
                tables.append(target_table)
                result_file = open(target_table + '_.csv', 'w')
                # CSV header, i.e. extract the column names
                header = line[line.find('(')+1:line.find(')')]
                header = re.sub(header_regex, ';', header)
                result_file.write(header)
                result_file.write('\n')
                result_file.close()
                result_file = open(target_table + '.csv', 'a')
                fast_forward = False
                continue
            # begin not yet found
            if fast_forward:
                continue
            # end of COPY reached
            if re.match(end_regex, line):
                line_count = 0
                fast_forward = True
                continue
            # convert tabs to ';' for CSV format
            newline = re.sub(line_regex, ';', line)
            result_file.write(newline)
    return tables

In [54]:
tables = dump_to_csv('public_postgresql/databases/PostgreSQL.sql')

dependencies: 0
gem_downloads: 0
linksets: 0
rubygems: 0
versions: 0


In [63]:
import pandas as pd

In [193]:
gems = pd.read_csv('rubygems.csv', ';', low_memory=False)

In [194]:
gems.drop('name', axis=1, inplace=True)
gems.drop('updated_at', axis=1, inplace=True)
gems.drop('downloads', axis=1, inplace=True)
gems.drop('slug', axis=1, inplace=True)
gems['created_at'] = pd.to_datetime(gems['created_at'])
gems = gems.set_index('created_at')
gems['day'] = gems.index.date
#gems.set_index('day', inplace=True)
gems.rename(columns=lambda x: x.replace('id', 'gems'), inplace=True)

In [188]:
# the actual magic :)
gems_cumsum = gems.groupby('day').count().cumsum()

In [189]:
gems_cumsum.to_csv('rubygems_cumsum.csv', ';')

In [190]:
import matplotlib

In [191]:
#gems_cumsum = pd.read_csv('counts.csv', ';')
plot = gems_cumsum.plot()
fig = plot.get_figure()
fig.savefig('rubygems.png')

In [192]:
deps = pd.read_csv('dependencies.csv', ';', low_memory=False)